In [4]:
from math import *
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPolygon,MultiLineString
from bokeh.sampledata.us_counties import data as counties
from TEN import TEN
TX = TEN('plot')
TX.read_bus()

Loading sub
Loading genbus
Loading bus2sub
Loading windfarm
Loading branches
Loading NRELsites
Done loading
Loading bus


In [3]:
#import bokeh
#bokeh.sampledata.download()

Creating U:\.bokeh directory
Creating U:\.bokeh\data directory
Using data directory: U:\.bokeh\data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3182088 bytes)
   3182088 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (5148539 bytes)
   5148539 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapminder_life_expectancy

In [2]:
# Calculate line lenght
#def greatCircleDistance(lat1, lon1, lat2, lon2):
#    R = 6371000
#    def haversin(x):
#        return sin(x/2)**2 
#    return 2*R * asin(sqrt(
#          haversin(lat2-lat1) +
#          cos(lat1) * cos(lat2) * haversin(lon2-lon1)))


In [5]:
# Calculate line lenght
def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    km = round(km, 3)

    return km
    #print(f"Distance: {meters} m")
    #print(f"Distance: {km} km")

In [6]:
# Set line lenght and voltage
TX.branches['dist'] = TX.branches.apply(lambda row : haversine([row['from_lat'],row['from_lon']],[row['to_lat'],row['to_lon']]), axis=1)
TX.branches['baseKV']=TX.bus.set_index('busID').reindex(TX.branches['from'])['baseKV'].values
TX.branches['withinCityStart'] = 'none'
TX.branches['withinCountyStart'] = 'none'
TX.branches['withinCityEnd'] = 'none'
TX.branches['withinCountyEnd'] = 'none'
TX.branches['withinZoneStart'] = 'none'
TX.branches['withinZoneEnd'] = 'none'

### Version A 

In [ ]:
# We check if the start and the end point of a line are in a city/county

In [7]:
# Check in which county/city a line is
gdf_nodes_start = gpd.GeoDataFrame(data={'x':TX.branches['from_lon'], 'y':TX.branches['from_lat']})
gdf_nodes_end = gpd.GeoDataFrame(data={'x':TX.branches['to_lon'], 'y':TX.branches['to_lat']})
# Load Texas cities, ranked by population
TX_cities = pd.read_csv(TX.data_loc+'TexasCities2015.txt',sep='\t')


In [8]:
for index, row in TX_cities.iterrows():
    place = row[1]
    gdf = ox.gdf_from_place({'city':place, 'state':'Texas'})
    print(gdf['place_name'])
    if (gdf.empty):
        print('not found')
    else:
        gdf_nodes_start.crs = gdf.crs
        gdf_nodes_start.name = 'nodes'
        gdf_nodes_start['geometry'] = gdf_nodes_start.apply(lambda row: Point((row['x'], row['y'])), axis=1)
        gdf_nodes_end.crs = gdf.crs
        gdf_nodes_end.name = 'nodes'
        gdf_nodes_end['geometry'] = gdf_nodes_end.apply(lambda row: Point((row['x'], row['y'])), axis=1)
        
        geometry = gdf['geometry'].iloc[0]
        if isinstance(geometry, Polygon):
            geometry = MultiPolygon([geometry])

        sindex_start = gdf_nodes_start.sindex
        possible_matches_index_start = list(sindex_start.intersection(geometry.bounds))
        possible_matches_start = gdf_nodes_start.iloc[possible_matches_index_start]
        precise_matches_start = possible_matches_start[possible_matches_start.intersects(geometry)]
        
        sindex_end = gdf_nodes_end.sindex
        possible_matches_index_end = list(sindex_end.intersection(geometry.bounds))
        possible_matches_end = gdf_nodes_end.iloc[possible_matches_index_end]
        precise_matches_end = possible_matches_end[possible_matches_end.intersects(geometry)]

        #intersect_index = precise_matches_start.index.intersection(precise_matches_end.index)
        TX.branches.loc[precise_matches_start.index,'withinCityStart'] = place
        TX.branches.loc[precise_matches_end.index,'withinCityEnd'] = place

0    Houston, Harris County, Texas, USA
Name: place_name, dtype: object
0    San Antonio, Bexar County, Texas, USA
Name: place_name, dtype: object
0    Dallas, Dallas County, Texas, USA
Name: place_name, dtype: object
0    Austin, Travis County, Texas, 78701, USA
Name: place_name, dtype: object
0    Fort Worth, Tarrant County, Texas, USA
Name: place_name, dtype: object
0    El Paso, El Paso County, Texas, USA
Name: place_name, dtype: object
0    Arlington, Tarrant County, Texas, USA
Name: place_name, dtype: object
0    Corpus Christi, Nueces County, Texas, USA
Name: place_name, dtype: object
0    Plano, Collin County, Texas, USA
Name: place_name, dtype: object
0    Laredo, Webb County, Texas, USA
Name: place_name, dtype: object
0    Lubbock, Lubbock County, Texas, USA
Name: place_name, dtype: object
0    Garland, Dallas County, Texas, USA
Name: place_name, dtype: object
0    Irving, Dallas County, Texas, USA
Name: place_name, dtype: object
0    Amarillo, Potter County, Texas, USA
Name:

In [7]:
Tx_county_list = list({county['name'] for code, county in counties.items() if county["state"] == "tx"})

In [8]:
for county in Tx_county_list:
    #print(county)
    gdf = ox.gdf_from_place({'county':county, 'state':'Texas'})
    print(gdf['place_name'])
    if (gdf.empty):
        print('not found')
    else:
        gdf_nodes_start.crs = gdf.crs
        gdf_nodes_start.name = 'nodes'
        gdf_nodes_start['geometry'] = gdf_nodes_start.apply(lambda row: Point((row['x'], row['y'])), axis=1)
        gdf_nodes_end.crs = gdf.crs
        gdf_nodes_end.name = 'nodes'
        gdf_nodes_end['geometry'] = gdf_nodes_end.apply(lambda row: Point((row['x'], row['y'])), axis=1)
        
        geometry = gdf['geometry'].iloc[0]
        if isinstance(geometry, Polygon):
            geometry = MultiPolygon([geometry])

        sindex_start = gdf_nodes_start.sindex
        possible_matches_index_start = list(sindex_start.intersection(geometry.bounds))
        possible_matches_start = gdf_nodes_start.iloc[possible_matches_index_start]
        precise_matches_start = possible_matches_start[possible_matches_start.intersects(geometry)]
        sindex_end = gdf_nodes_end.sindex
        possible_matches_index_end = list(sindex_end.intersection(geometry.bounds))
        possible_matches_end = gdf_nodes_end.iloc[possible_matches_index_end]
        precise_matches_end = possible_matches_end[possible_matches_end.intersects(geometry)]

        #intersect_index = precise_matches_start.index.intersection(precise_matches_end.index)
        TX.branches.loc[precise_matches_start.index,'withinCountyStart'] = county
        TX.branches.loc[precise_matches_end.index,'withinCountyEnd'] = county

0    Donley County, Texas, United States of America
Name: place_name, dtype: object
0    Gillespie County, Texas, United States of America
Name: place_name, dtype: object
0    Hopkins County, Texas, United States of America
Name: place_name, dtype: object
0    Williamson County, Texas, United States of Ame...
Name: place_name, dtype: object
0    Coke County, Texas, United States of America
Name: place_name, dtype: object
0    Fayette County, Texas, United States of America
Name: place_name, dtype: object
0    Nueces County, Texas, United States of America
Name: place_name, dtype: object
0    Castro County, Texas, United States of America
Name: place_name, dtype: object
0    Jim Hogg County, Texas, 78361, United States o...
Name: place_name, dtype: object
0    Reagan County, Texas, 76932, United States of ...
Name: place_name, dtype: object
0    Hill County, Texas, United States of America
Name: place_name, dtype: object
0    Hutchinson County, Texas, United States of Ame...
Name: place

0    Tom Green County, Texas, United States of America
Name: place_name, dtype: object
0    Duval County, Texas, United States of America
Name: place_name, dtype: object
0    Martin County, Texas, United States of America
Name: place_name, dtype: object
0    Angelina County, Texas, United States of America
Name: place_name, dtype: object
0    Parker County, Texas, United States of America
Name: place_name, dtype: object
0    Deaf Smith County, Texas, United States of Ame...
Name: place_name, dtype: object
0    McLennan County, Texas, United States of America
Name: place_name, dtype: object
0    Bosque County, Texas, United States of America
Name: place_name, dtype: object
0    Edwards County, Texas, 78880, United States of...
Name: place_name, dtype: object
0    Coryell County, Texas, United States of America
Name: place_name, dtype: object
0    Hunt County, Texas, United States of America
Name: place_name, dtype: object
0    Rains County, Texas, United States of America
Name: place_na

0    Wilson County, Texas, United States of America
Name: place_name, dtype: object
0    Henderson County, Texas, United States of America
Name: place_name, dtype: object
0    Harris County, Texas, United States of America
Name: place_name, dtype: object
0    Comanche County, Texas, United States of America
Name: place_name, dtype: object
0    Limestone County, Texas, United States of America
Name: place_name, dtype: object
0    San Patricio County, Texas, United States of A...
Name: place_name, dtype: object
0    Bee County, Texas, United States of America
Name: place_name, dtype: object
0    Hockley County, Texas, United States of America
Name: place_name, dtype: object
0    Matagorda County, Texas, United States of America
Name: place_name, dtype: object
0    Gonzales County, Texas, United States of America
Name: place_name, dtype: object
0    Polk County, Texas, United States of America
Name: place_name, dtype: object
0    Shelby County, Texas, United States of America
Name: place_

In [9]:
weatherZones = pd.read_csv(TX.data_loc + "ERCOTWeatherZones.csv")

In [10]:
for zone in weatherZones.columns:
    TX.branches.loc[TX.branches['withinCountyStart'].isin(weatherZones[zone]),'withinZoneStart'] = zone
    TX.branches.loc[TX.branches['withinCountyEnd'].isin(weatherZones[zone]),'withinZoneEnd'] = zone

In [12]:
TX.branches.columns

Index(['from', 'to', 'r', 'x', 'b', 'Capacity', 'rateB', 'rateC', 'ratio',
       'angle', 'status', 'angmin', 'angmax', 'Pf', 'Qf', 'Pt', 'Qt', 'mu_Sf',
       'mu_St', 'mu_angmin', 'mu_angmax', 'from_lat', 'from_lon', 'to_lat',
       'to_lon', 'from_x', 'from_y', 'to_x', 'to_y', 'dist', 'baseKV',
       'withinCityStart', 'withinCountyStart', 'withinCityEnd',
       'withinCountyEnd', 'withinZoneStart', 'withinZoneEnd'],
      dtype='object')

In [11]:
TX.branches.to_csv('branchTest')

### Version B 

In [ ]:
# This version will categorize lines of dist=0 as not existant. That means when checking a line with zero distance the method 
# will return say that the line is not inside a shape. This is oposit to version A where we check the start and the endpoint
# location

In [ ]:
# This code writes the branches as LineStrings into geopandas. These objects can be used to figure out geometrical relations
from_bus = TX.branches[['from_lon','from_lat']].values.tolist()
to_bus = TX.branches[['to_lon','to_lat']].values.tolist()
[fbx,fby] =zip(*from_bus)
[tbx,tby] =zip(*to_bus)
from_zip = zip(fbx,fby)
to_zip = zip(tbx,tby)
TX_MultiLines = MultiLineString(list(zip(from_zip,to_zip)))
TX_lines = gpd.GeoSeries(data = TX_MultiLines)

In [19]:
for index, row in TX_cities.iterrows():
    place = row[1]
    print(place)
    gdf = ox.gdf_from_place(place + ', Texas, USA')
    if (gdf.empty):
        print('not found')
    else:
        TX.branches.loc[TX_lines.apply(gdf.contains)[0],'withinCity'] = place

Houston
San Antonio
Dallas
Austin
Fort Worth
El Paso
Arlington
Corpus Christi
Plano
Laredo
Lubbock
Garland
Irving
Amarillo
Grand Prairie
Brownsville
McKinney
Frisco
Pasadena
Mesquite
Killeen
McAllen
Carrollton
Midland
Waco
Denton
Abilene
Odessa
Beaumont
Round Rock
The Woodlands (CDP)
Richardson
Pearland
College Station
Wichita Falls
Lewisville
Tyler
San Angelo
League City
Allen
Sugar Land
Edinburg
Mission
Longview
Bryan
Pharr
Baytown
Missouri City
Temple
Flower Mound
New Braunfels
North Richland Hills
Conroe
Victoria
Cedar Park
Harlingen
Atascocita (CDP)
not found
Mansfield
Georgetown
San Marcos
Rowlett
Pflugerville
Port Arthur
Spring (CDP)
Euless
DeSoto
Grapevine
Galveston


In [24]:
TX.branches[TX.branches['withinCity'] != 'none'].size

24004

In [15]:
TX.branches['withinCounty'].unique().size

109

In [17]:
len(Tx_county_list)

254